In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import cv2
import csv
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import gc

from typing import List, Dict
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset

In [4]:
classes = []
with open("/content/drive/MyDrive/Colab_Notebooks/mini/category.csv", 'r') as csvfile:
  # creating a csv reader object
  csvreader = csv.reader(csvfile)
  _ = next(csvreader)
  for i in csvreader:
    classes.append(i[1])

In [5]:
files = []
with open("/content/drive/MyDrive/Colab_Notebooks/mini/train_filename.csv", 'r') as f:
  csvreader = csv.reader(f)
  _ = next(csvreader)
  for i in csvreader:
    files.append(i)

In [6]:
!unzip '/content/drive/MyDrive/Colab_Notebooks/mini/train_crop.zip' -d '/content'

串流輸出內容已截斷至最後 5000 行。
  inflating: /content/train_crop/66940.jpg  
  inflating: /content/train_crop/66941.jpg  
  inflating: /content/train_crop/66943.jpg  
  inflating: /content/train_crop/66944.jpg  
  inflating: /content/train_crop/66945.jpg  
  inflating: /content/train_crop/66946.jpg  
  inflating: /content/train_crop/66948.jpg  
  inflating: /content/train_crop/66949.jpg  
  inflating: /content/train_crop/6695.jpg  
  inflating: /content/train_crop/66950.jpg  
  inflating: /content/train_crop/66951.jpg  
  inflating: /content/train_crop/66952.jpg  
  inflating: /content/train_crop/66953.jpg  
  inflating: /content/train_crop/66954.jpg  
  inflating: /content/train_crop/66955.jpg  
  inflating: /content/train_crop/66956.jpg  
  inflating: /content/train_crop/66957.jpg  
  inflating: /content/train_crop/66959.jpg  
  inflating: /content/train_crop/66960.jpg  
  inflating: /content/train_crop/66961.jpg  
  inflating: /content/train_crop/66963.jpg  
  inflating: /content/train_crop/66

In [7]:
img_dataset = []
label_dataset = []
img_path = "/content/train_crop/"
for i in files:
  img = cv2.imread(img_path + i[0])
  img = cv2.resize(img, (224, 224))
  img = img.astype("float32") / 255.0
  img_dataset.append(torch.from_numpy(img))
  label_dataset.append(int(i[1]))
  print(i)

串流輸出內容已截斷至最後 5000 行。
['66940.jpg', '93']
['66941.jpg', '58']
['66943.jpg', '95']
['66944.jpg', '62']
['66945.jpg', '17']
['66946.jpg', '97']
['66948.jpg', '19']
['66949.jpg', '4']
['6695.jpg', '27']
['66950.jpg', '15']
['66951.jpg', '74']
['66952.jpg', '43']
['66953.jpg', '18']
['66954.jpg', '96']
['66955.jpg', '5']
['66956.jpg', '89']
['66957.jpg', '93']
['66959.jpg', '41']
['66960.jpg', '44']
['66961.jpg', '68']
['66963.jpg', '18']
['66964.jpg', '49']
['66967.jpg', '55']
['66968.jpg', '34']
['6697.jpg', '72']
['66970.jpg', '87']
['66973.jpg', '25']
['66974.jpg', '94']
['66977.jpg', '65']
['66978.jpg', '70']
['66980.jpg', '32']
['66981.jpg', '61']
['66982.jpg', '89']
['66984.jpg', '29']
['66985.jpg', '82']
['66987.jpg', '2']
['66988.jpg', '91']
['6699.jpg', '71']
['66990.jpg', '64']
['66991.jpg', '35']
['66992.jpg', '52']
['66994.jpg', '24']
['66996.jpg', '29']
['66997.jpg', '37']
['66998.jpg', '91']
['66999.jpg', '92']
['6700.jpg', '68']
['67001.jpg', '65']
['67002.jpg', '33']
['6700

In [8]:
tmp = torch.randn(224, 224, 3)
for i in range(len(img_dataset)):
  if img_dataset[i].shape != tmp.shape:
    print(img_dataset[i].shape)

In [9]:
print(f'Can I can use GPU now? -- {torch.cuda.is_available()}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Can I can use GPU now? -- True


In [10]:
class trainDataset(Dataset):
  def __init__(self, dataset, labels, transform):
    self.data = dataset
    self.labels = labels
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    if self.transform:
      self.data[index] = self.transform(self.data[index])
    return self.data[index], self.labels[index]

In [11]:
def collate_batch(batch_list):
  assert type(batch_list) == list, f"Error"
  batch_size = len(batch_list)
  data = [item[0] for item in batch_list]
  labels = torch.cat([item[1].unsqueeze(0) for item in batch_list], dim=0).reshape(1, batch_size, -1)
  return data, labels

In [12]:
batch_size_train = 124

# print(img_dataset[0].shape)
for i in range(len(img_dataset)):
  img_dataset[i] = img_dataset[i].permute(2, 0, 1)

train_data = img_dataset[:len(img_dataset)-10000]
train_label = label_dataset[:len(img_dataset)-10000]
test_data = img_dataset[len(img_dataset)-10000:]
test_label = label_dataset[len(img_dataset)-10000:]

In [13]:
transform_train = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
transform_test = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_dataset = trainDataset(train_data, torch.tensor(train_label), transform = transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True,
                                           num_workers=0, collate_fn = collate_batch)

test_dataset = trainDataset(test_data, torch.tensor(test_label), transform = transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size_train, shuffle=True,
                                          num_workers=0, collate_fn = collate_batch)

In [14]:
resnet50 = models.resnet50(pretrained=True)
resnet50.fc = nn.Linear(resnet50.fc.in_features, 100)
resnet50 = resnet50.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 181MB/s]


In [15]:
from itertools import accumulate
def train(model: nn.Module,
          loss_fn: nn.modules.loss._Loss,
          optimizer: torch.optim.Optimizer,
          train_loader: torch.utils.data.DataLoader,
          epoch: int=0)-> List:
    # ----------- <Your code> ---------------
    model.train()
    train_loss = []
    train_accuracy = []

    for batch_idx, (images, targets) in enumerate(train_loader):
      images = torch.stack(images)
      # images = images.permute(0, 3, 1, 2)
      images = images.to(device)
      targets = targets.to(device)
      targets = targets.view(-1)
      optimizer.zero_grad()
      output = model(images)
      loss = loss_fn(output, targets)
      train_loss.append(loss.item())
      loss.backward()
      optimizer.step()

      output_class = torch.argmax(torch.softmax(output, dim=1), dim=1)
      accuracy = (output_class == targets).sum().item() / len(targets)
      train_accuracy.append(accuracy)

      if batch_idx % (len(train_loader)//8) == 0:
        print(f'Epoch {epoch}: [{batch_idx*len(images)}/{len(train_loader.dataset)}] Loss: {loss.item():.3f}')

      images = []
      targets = []
      output = []
      loss = []
      output_class = []
      torch.cuda.empty_cache()
      gc.collect()

    # ----------- <End Your code> ---------------
    ave_loss = sum(train_loss) / len(train_loss)
    print(f'Average loss on epoch {epoch}: {ave_loss:.3f}')
    ave_acc = sum(train_accuracy) / len(train_accuracy)
    print(f'Average accuracy on epoch {epoch}: {ave_acc:.3f}')
    return ave_loss, ave_acc

In [16]:
def test(model: nn.Module,
         loss_fn: nn.modules.loss._Loss,
         test_loader: torch.utils.data.DataLoader,
         epoch: int=0)-> Dict:
    # ----------- <Your code> ---------------
    model.eval()
    l = 0
    correct = 0
    preds = torch.empty((0)).to(device)

    with torch.no_grad():
      for images, targets in test_loader:
        images = torch.stack(images)
        # images = images.permute(0, 3, 1, 2)
        images = images.to(device)
        targets = targets.to(device)
        targets = targets.view(-1)
        output = model(images)
        l += loss_fn(output, targets).item() # this is the average of the batch(1000)
        pred = output.data.max(1, keepdim=True)[1]
        preds = torch.cat((preds, pred), dim = 0)
        correct += pred.eq(targets.data.view_as(pred)).sum()

    l /= len(test_loader) # no need to devide the whole data: 10000, devide len(test_loader): 10
    accur = correct / len(test_loader.dataset)
    test_stat = dict(loss = l, accuracy = accur, prediction = preds)
    total_num = len(test_loader.dataset)

    print(f"Test result on epoch {epoch}: total sample: {total_num}, Avg loss: {test_stat['loss']:.3f}, Acc: {100*test_stat['accuracy']:.3f}%")
    # ----------- <End Your code> ---------------
    # dictionary output should include loss, accuracy and prediction
    assert "loss" and "accuracy" and "prediction" in test_stat.keys()
    # "prediction" should be a 1D tensor
    assert len(test_stat["prediction"]) == len(test_loader.dataset)
    assert isinstance(test_stat["prediction"], torch.Tensor)
    return test_stat

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
start = time.time()
max_epoch = 10
optimizer3 = optim.SGD(resnet50.parameters(), lr=0.01, momentum=0.8)
model_path = '/content/drive/MyDrive/Colab_Notebooks/mini/'

for epoch in range(max_epoch):
  loss, accuracy = train(resnet50, criterion, optimizer3, train_loader, epoch)
  print("Loss", loss, "Accuracy", accuracy)
  torch.save(resnet50.state_dict(), model_path + "model_" + str(epoch) + ".pth")

  gc.collect()

  test_stat = test(resnet50, criterion, test_loader, epoch)
  print("Test", test_stat)

end = time.time()
print(f'Finished Training after {end-start} s ')

Epoch 0: [0/44483] Loss: 1.547
Epoch 0: [5456/44483] Loss: 1.615
Epoch 0: [10912/44483] Loss: 1.467
Epoch 0: [16368/44483] Loss: 1.449
Epoch 0: [21824/44483] Loss: 1.442
Epoch 0: [27280/44483] Loss: 1.609
Epoch 0: [32736/44483] Loss: 1.761
Epoch 0: [38192/44483] Loss: 1.622
Epoch 0: [43648/44483] Loss: 1.100
Average loss on epoch 0: 1.508
Average accuracy on epoch 0: 0.685
Loss 1.5076211601246698 Accuracy 0.6850844590758324


In [1]:
torch.cuda.empty_cache()
# Check the current GPU memory usage
current_memory = torch.cuda.memory_allocated()
print("Current GPU memory usage:", current_memory, "bytes")

# Check the maximum GPU memory usage
max_memory = torch.cuda.max_memory_allocated()
print("Maximum GPU memory usage:", max_memory, "bytes")

torch.cuda.reset_max_memory_allocated()
gc.collect()

NameError: name 'torch' is not defined